In [ ]:
%pip uninstall torch

In [ ]:
%pip install torch==2.3.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#%pip install torch torchvision torchaudio
#%pip install torch==1.10.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

%pip install opencv-python
%pip install scikit-learn
%pip install yolov5


In [ ]:
import os

# YOLOv5 클론 및 설치
!git clone https://github.com/ultralytics/yolov5
os.chdir('yolov5')
!pip install -r requirements.txt


In [ ]:
import zipfile
import os

# 압축 파일 경로와 해제 경로 설정
zip_file_path = "./face_mask.zip"
extract_path = "./face_mask"

# 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("데이터셋이 압축 해제되었습니다.")

# 압축 해제 후 파일 목록 확인
for root, dirs, files in os.walk(extract_path):
    print(f"디렉토리 경로: {root}")
    print(f"디렉토리 내 폴더: {dirs}")
    print(f"디렉토리 내 파일: {files}")


In [ ]:
import os

# 데이터셋 경로
dataset_path = extract_path

# 이미지 파일과 라벨 파일 목록 생성
image_files = []
label_files = []

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'):
            image_files.append(os.path.join(root, file))
        if file.endswith('.txt'):
            label_files.append(os.path.join(root, file))

# 목록 출력
print(f"이미지 파일 목록: {image_files}")
print(f"라벨 파일 목록: {label_files}")


In [ ]:
import shutil
from sklearn.model_selection import train_test_split

# 데이터셋 분할
train_images, temp_images = train_test_split(image_files, test_size=0.4, random_state=42)
val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

# 분할된 데이터셋을 저장할 디렉토리 생성
os.makedirs(os.path.join(dataset_path, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'val/labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'test/images'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'test/labels'), exist_ok=True)

# 파일 이동 함수
def move_files(file_list, src_dir, dest_dir):
    for file_path in file_list:
        file_name = os.path.basename(file_path)
        shutil.move(file_path, os.path.join(dest_dir, file_name))
        label_name = file_name.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(src_dir, label_name)
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_dir.replace('images', 'labels'), label_name))

# 데이터셋 분할 후 파일 이동
move_files(train_images, dataset_path, os.path.join(dataset_path, 'train/images'))
move_files(val_images, dataset_path, os.path.join(dataset_path, 'val/images'))
move_files(test_images, dataset_path, os.path.join(dataset_path, 'test/images'))

print("데이터셋이 train, validation, test로 분할되었습니다.")


In [ ]:
data_config = """
train: ./face_mask/train/images
val: ./face_mask/val/images
test: ./face_mask/test/images

nc: 3  #class count
names: ['mask_worn_correctly', 'mask_not_worn', 'mask_worn_incorrectly']
"""

with open('./face_mask/data.yaml', 'w') as f:
    f.write(data_config)

print("data.yaml 파일이 생성되었습니다.")


In [ ]:
hyp_config = """
lr0: 0.01
lrf: 0.1
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05
cls: 0.5
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
iou_t: 0.2
anchor_t: 4.0
fl_gamma: 0.0
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
copy_paste: 0.0
"""

with open('./face_mask/hyp.yaml', 'w') as f:
    f.write(hyp_config)

print("hyp.yaml 파일이 생성되었습니다.")


In [ ]:
import torch
import yaml
from yolov5 import train

# 데이터셋 구성 파일 생성
data_config = {
    'path': dataset_path,
    'train': os.path.join(dataset_path, 'train/images'),
    'val': os.path.join(dataset_path, 'val/images'),
    'nc': 3,
    'names': ['Wearing mask well', 'Not wearing mask', 'Wearing mask incorrectly']
}

with open(os.path.join(dataset_path, 'data.yaml'), 'w') as outfile:
    yaml.dump(data_config, outfile, default_flow_style=False)

# 학습 파라미터 설정
model_config = 'yolov5s.yaml'  # YOLOv5 모델 구성 파일
data_config_path = os.path.join(dataset_path, 'data.yaml')
epochs = 50
batch_size = 16
img_size = 416
results_dir = os.path.join(dataset_path, "results")

# 모델 학습 및 결과 저장
train.run(imgsz=img_size, batch_size=batch_size, epochs=epochs, data=data_config_path, cfg=model_config, project=results_dir, name="exp")

print(f"모델 학습이 완료되었습니다. 결과는 {results_dir}/exp 디렉토리에 저장되었습니다.")


In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 50 --data C:/Users/user/Desktop/ai_work/face_mask/data.yaml --hyp C:/Users/user/Desktop/ai_work/face_mask/hyp.yaml --weights yolov5s.pt --cache


In [ ]:
import os
import matplotlib.pyplot as plt

# 훈련 결과가 저장된 기본 디렉토리
base_dir = "./face_mask/train"

# 디렉토리 목록 출력
if os.path.exists(base_dir):
    exp_dirs = os.listdir(base_dir)
    print(f"훈련 결과 디렉토리 목록: {exp_dirs}")

    # results.txt 파일 경로 찾기
    log_file = None
    for exp_dir in exp_dirs:
        exp_path = os.path.join(base_dir, exp_dir)
        files = os.listdir(exp_path)
        if 'results.txt' in files:
            log_file = os.path.join(exp_path, 'results.txt')
            break

    if log_file:
        print(f"'results.txt' 파일 경로: {log_file}")
        results = []

        # 결과 파일 읽기
        with open(log_file, 'r') as f:
            for line in f:
                results.append([float(value) for value in line.split()])

        results = list(zip(*results))

        # 그래프 그리기
        plt.figure(figsize=(10, 5))

        # 손실 함수 변화 그래프
        plt.subplot(1, 2, 1)
        plt.plot(results[0], results[1], label='Box Loss')
        plt.plot(results[0], results[2], label='Objectness Loss')
        plt.plot(results[0], results[3], label='Classification Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss over Epochs')
        plt.legend()

        # 성능 지표 그래프
        plt.subplot(1, 2, 2)
        plt.plot(results[0], results[4], label='mAP@0.5')
        plt.plot(results[0], results[5], label='Precision')
        plt.plot(results[0], results[6], label='Recall')
        plt.xlabel('Epoch')
        plt.ylabel('Metric')
        plt.title('Performance Metrics over Epochs')
        plt.legend()

        plt.tight_layout()
        plt.show()
    else:
        print("'results.txt' 파일을 찾을 수 없습니다.")
else:
    print("훈련 결과 디렉토리가 존재하지 않습니다. 훈련이 정상적으로 완료되었는지 확인하세요.")
